In [1]:
import os
from pathlib import Path

import torch
from cliport import agents
from cliport.dataset import RavensDataset, RavensMultiTaskDataset

import hydra
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from cliport.utils import utils

os.environ['CLIPORT_ROOT'] = '/sfs/weka/scratch/ys5hd/cliport'

/home/ys5hd/.conda/envs/pytorch_yash/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_hip.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Load configs
ROOT_DIR = os.environ['CLIPORT_ROOT']
config_file = 'train.yaml' 
cfg = utils.load_hydra_config(os.path.join(ROOT_DIR, f'cliport/cfg/{config_file}'))

In [3]:
wandb_logger = WandbLogger(name=cfg['tag']) if cfg['train']['log'] else None

In [4]:
# Set False for cliport architecture
# Set the as task name for Text pre-processed Architecture

LLM_PARSING = 'align-rope'

TRAIN_TASK = 'align-rope'
#'put-block-in-bowl-seen-colors'
#'packing-boxes-pairs-seen-colors'
#'stack-block-pyramid-seq-unseen-colors'

TRAIN_AGENT = 'cliport'
TRAIN_NDEMOS = 100
if LLM_PARSING:
    EXPS = 'exps_llm'
else:
    EXPS = 'exps'

cfg['train']['task'] = TRAIN_TASK
cfg['train']['agent'] = TRAIN_AGENT
cfg['train']['attn_stream_fusion_type'] = 'add'
cfg['train']['trans_stream_fusion_type'] = 'conv'
cfg['train']['lang_fusion_type'] = 'mult'
cfg['train']['n_demos'] = 100
cfg['train']['n_steps'] =  101000 #201000
cfg['train']['exp_folder'] = EXPS
cfg['dataset']['cache'] = False

In [5]:
cfg['train']['train_dir'] = f'{ROOT_DIR}/{EXPS}/{TRAIN_TASK}-{TRAIN_AGENT}-n{TRAIN_NDEMOS}-train'

In [6]:
hydra_dir = Path(os.getcwd())
checkpoint_path = os.path.join(cfg['train']['train_dir'], 'checkpoints')
last_checkpoint_path = os.path.join(checkpoint_path, 'last.ckpt')

last_checkpoint = last_checkpoint_path if os.path.exists(last_checkpoint_path) and cfg['train']['load_from_last_ckpt'] else None
checkpoint_callback = ModelCheckpoint(
    monitor=cfg['wandb']['saver']['monitor'],
    filepath=os.path.join(checkpoint_path, 'best'),
    save_top_k=1,
    save_last=True,
)

In [7]:
max_epochs = cfg['train']['n_steps'] // cfg['train']['n_demos']

In [8]:
trainer = Trainer(
    gpus=cfg['train']['gpu'],
    fast_dev_run=cfg['debug'],
    logger=wandb_logger,
    checkpoint_callback=checkpoint_callback,
    max_epochs=max_epochs,
    automatic_optimization=False,
    check_val_every_n_epoch=max_epochs // 50,
    resume_from_checkpoint=last_checkpoint,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [9]:
if last_checkpoint:
    print(f"Resuming: {last_checkpoint}")
    last_ckpt = torch.load(last_checkpoint)
    trainer.current_epoch = last_ckpt['epoch']
    trainer.global_step = last_ckpt['global_step']
    del last_ckpt

Resuming: /sfs/weka/scratch/ys5hd/cliport/exps_llm/align-rope-cliport-n100-train/checkpoints/last.ckpt


In [10]:
DATA_DIR = "/sfs/weka/scratch/ys5hd/cliport/cliport/data/"

In [11]:
# Config
data_dir = DATA_DIR
task = cfg['train']['task']
agent_type = cfg['train']['agent']
n_demos = cfg['train']['n_demos']
n_val = cfg['train']['n_val']
name = '{}-{}-{}'.format(task, agent_type, n_demos)

In [12]:
# Datasets
dataset_type = cfg['dataset']['type']
if 'multi' in dataset_type:
    train_ds = RavensMultiTaskDataset(data_dir, cfg, group=task, mode='train', n_demos=n_demos, augment=True)
    val_ds = RavensMultiTaskDataset(data_dir, cfg, group=task, mode='val', n_demos=n_val, augment=False)
else:
    train_ds = RavensDataset(os.path.join(data_dir, '{}-train'.format(task)), cfg, n_demos=n_demos, augment=True)
    val_ds = RavensDataset(os.path.join(data_dir, '{}-val'.format(task)), cfg, n_demos=n_val, augment=False)

In [13]:
# Initialize agent
agent = agents.names[agent_type](name, cfg, train_ds, val_ds, llm_parsing=LLM_PARSING)

Attn FCN - Stream One: plain_resnet_lat, Stream Two: clip_lingunet_lat, Stream Fusion: add
Transport FCN - Stream One: plain_resnet_lat, Stream Two: clip_lingunet_lat, Stream Fusion: conv
Agent: align-rope-cliport-100, Logging: False


In [14]:
# Main training loop
trainer.fit(agent)

/home/ys5hd/.conda/envs/pytorch_yash/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  warnings.warn(*args, **kwargs)
Set SLURM handle signals.

  | Name      | Type                            | Params
--------------------------------------------------------------
0 | attention | TwoStreamAttentionLangFusionLat | 194 M 
1 | transport | TwoStreamTransportLangFusionLat | 388 M 


Validation sanity check: 0it [00:00, ?it/s]

pybullet build time: Sep 22 2020 00:56:01

Attn Err - Dist: 89.00, Theta: 0.00
Transport Err - Dist: 238.00, Theta: 0.00


/home/ys5hd/.conda/envs/pytorch_yash/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

...


Attn Err - Dist: 1765.00, Theta: 0.00
Transport Err - Dist: 4543.00, Theta: 6.81


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Validating: 0it [00:00, ?it/s]


Attn Err - Dist: 1817.00, Theta: 0.00
Transport Err - Dist: 6195.00, Theta: 5.59


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

